In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from tqdm import tqdm

In [3]:
import logging
import math
import os
import random
import sys
import time
import pickle
import copy

import numpy as np
import torch

from fairseq import (
    checkpoint_utils,
    distributed_utils,
    options,
    quantization_utils,
    tasks,
    utils,
)
from fairseq.data import iterators
from fairseq.logging import meters, metrics, progress_bar
from fairseq.trainer import Trainer
from fairseq.model_parallel.megatron_trainer import MegatronTrainer
from fairseq.models.pruned_transformer import PrunedTransformerModel

In [4]:
args = pickle.load(open("argsfile.p", "rb"))

In [5]:
print(args)

Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='pruned_transformer_vaswani_wmt_en_de_big', attention_dropout=0.0, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin/wmt16_en_de_bpe32k', data_buffer_size=2, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=16, decoder_embed_dim=1024, decoder_embed_path=None, decoder_ffn_embed_dim=4096, decoder_input_dim=1024, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=1024, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=

In [6]:
task = tasks.setup_task(args)


In [7]:
task.load_dataset(args.valid_subset, combine=False, epoch=1)
dataset = task.dataset(args.valid_subset)


In [8]:
checkpoint_dir = "/home/raj/data/raj-learn/checkpoints/lr-rewind_0.75sparsity_0.2frac_30epochs/"
model_paths = ["checkpoint_LTH0_epoch60.pt",
              "checkpoint_LTH1_epoch60_sparsity0.168.pt",
              "checkpoint_LTH2_epoch60_sparsity0.302.pt",
              "checkpoint_LTH3_epoch60_sparsity0.410.pt", 
              "checkpoint_LTH4_epoch60_sparsity0.496.pt", 
              "checkpoint_LTH5_epoch60_sparsity0.565.pt",
              "checkpoint_LTH6_epoch60_sparsity0.620.pt",
              "checkpoint_LTH7_epoch60_sparsity0.664.pt"]


In [9]:
args.path = checkpoint_dir + model_paths[0]

In [10]:
models, _model_args = checkpoint_utils.load_model_ensemble(
        args.path.split(os.pathsep),
        task=task,
    )
model = models[0]
model.cuda()
model.eval()

PrunedTransformerModel(
  (encoder): TransformerEncoder(
    (embed_tokens): Embedding(32768, 1024, padding_idx=1)
    (embed_positions): SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (k_proj): 

In [ ]:
all_hidden = {}
for batch in tqdm(itr):
    ids = batch["id"].cpu().numpy().tolist()
    src_lens = batch["net_input"]["src_lengths"].cpu().numpy()
    enc_outputs = model.encoder(batch["net_input"]["src_tokens"].cuda(), batch["net_input"]["src_lengths"].cuda(), 
                                        return_all_hiddens=False, return_all_attns=True)
    #print(enc_outputs.encoder_self_attns[0].shape)
    encenc_attns = [x.detach().cpu().numpy() for x in enc_outputs.encoder_self_attns]
    del enc_outputs
    
    out, props = model(batch["net_input"]["src_tokens"].cuda(), batch["net_input"]["src_lengths"].cuda(), \
                        batch["net_input"]["prev_output_tokens"].cuda())
    encdec_attns = [x.detach().cpu().numpy() for x in props["encdec_attns"]]
    #print(encdec_attns[0].shape)
    decdec_attns = [x.detach().cpu().numpy() for x in props["decdec_attns"]]
    #print(decdec_attns[0].shape)

    print(batch["net_input"]["prev_output_tokens"].shape)
    print(batch["net_input"]["prev_output_tokens"][12,:])
    print(batch["target"][12,:])
    del out, props
    del batch

    for i, id_ in enumerate(ids):
        all_attns = {"encenc": [], "encdec": [], "decdec": []}
        for attn in encenc_attns:
            all_attns["encenc"].append(attn[i,:,:src_lens[i],:src_lens[i]])
        for attn in encdec_attns:
            all_attns["encdec"].append(attn[:,i,:src_lens[i],:])
            print(attn[3,i,:,:])
        for attn in decdec_attns:
            all_attns["decdec"].append(attn[i,:,:,:])
            #print(attn[i,3,:,:])
            break
        all_masks[id_] = all_attns
        
    break